In [35]:
import pandas as pd
import os
from all_aircrafts import all_aircrafts
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [36]:
import folium
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster

import plotly.express as px
import plotly.figure_factory as ff

## Planespotter data

After analysing the task at hand of identifying if some airlines are better to fly with than other we decided that we needed to get our hands on the fleets of each airline. To do that we contacted planespotter.net

In [37]:
path = os.path.join(".", "data", "planespotter", "2019-11-12 prodlist.xlsx")
planespotter_aircrafts = pd.read_excel(path)

In [38]:
planespotter_aircrafts.head()

,Aircraft Manufacturer,Aircraft Type,Operator,Operator ID,Operator Category,Operator Country,Operator Parent/Group,Parent/Group ID,Parent/Group Country,First Operator,...,Build Country,First Customer Delivery Date,Delivery Date Operator,Exit Date Operator,Nr. of Engines,Engine Manufacturer,Engine Type,Config (Pax/Combi/Cargo/Other),Seat Total,Age (Years)
0,Airbus,Airbus A300B1,Airbus Industrie,1336,Manufacturer,France,NaN,NaN,NaN,NaN,...,France,NaN,1972-01-01,NaN,2,General Electric,CF6-50,Other,0,1.7
1,Airbus,Airbus A300B1,TEA - Trans European Airlines,294,Airline,Belgium,NaN,NaN,NaN,TEA - Trans European Airlines,...,France,1974-11-01,1974-11-01,NaN,2,General Electric,CF6-50,NaN,0,30.4
2,Airbus,Airbus A300B2,SA Novespace,7216,Other,France,NaN,NaN,NaN,SA Novespace,...,France,2005-01-01,2005-01-01,NaN,2,General Electric,CF6-50,NaN,0,46.4
3,Airbus,Airbus A300B2,Air Inter,1309,Airline,France,NaN,NaN,NaN,Air Inter,...,France,1977-01-01,1977-01-01,NaN,2,General Electric,CF6-50,NaN,0,19.4
4,Airbus,Airbus A300B2,Air France,1306,Airline,France,Air France-KLM,11321.0,France,Air France,...,France,1974-05-01,1974-05-01,NaN,2,General Electric,CF6-50,NaN,0,23.1


In [39]:
planespotter_aircrafts_dictionnary = planespotter_aircrafts.groupby("Aircraft Type").count()
planespotter_aircrafts_dictionnary = planespotter_aircrafts_dictionnary.reset_index()
planespotter_aircrafts_dictionnary["Wiki name"] = planespotter_aircrafts_dictionnary["Aircraft Type"]
planespotter_aircrafts_dictionnary = planespotter_aircrafts_dictionnary[["Aircraft Type", "Wiki name"]]
planespotter_aircrafts_dictionnary = planespotter_aircrafts_dictionnary.rename(columns={"Aircraft Type": "Planespotter name"})
planespotter_aircrafts_dictionnary.head()

,Planespotter name,Wiki name
0,ATR 42,ATR 42
1,ATR 72,ATR 72
2,Airbus A220-100,Airbus A220-100
3,Airbus A220-300,Airbus A220-300
4,Airbus A300-600,Airbus A300-600


In [40]:
all_aircrafts = all_aircrafts.reset_index()
all_aircrafts = all_aircrafts.drop('index', axis=1)
all_aircrafts

,Model,Seats,First flight,Fuel burn kg/km,Fuel per seat L/100km,Sector km,CO2 per seat kg/100km,CO2 kg/km,Aircraft type
0,Antonov An-148 (241 nmi),89,2004,4.23,5.95,560.0,13.187973,13.05378,commuter_aircraft
1,Antonov An-158 (241 nmi),99,2010,4.34,5.47,560.0,12.124069,13.39324,commuter_aircraft
2,ATR 42-500,48,1995,1.26,3.15,560.0,6.981868,3.88836,commuter_aircraft
3,ATR 72-500,70,1997,1.42,2.53,560.0,5.607659,4.38212,commuter_aircraft
4,Beechcraft 1900D (226 nm),19,1982,1.00,6.57,560.0,14.562182,3.08600,commuter_aircraft
...,...,...,...,...,...,...,...,...,...
128,Boeing 777-9X,395,2020,9.04,2.85,13300.0,6.316928,27.89744,long_haul_aircraft
129,Boeing 787-8,243,2011,5.38,2.77,8610.0,6.139611,16.60268,long_haul_aircraft
130,Boeing 787-9,294,2013,5.85,2.49,8610.0,5.519000,18.05310,long_haul_aircraft
131,Boeing 787-9,304,2013,5.63,2.31,9208.0,5.120036,17.37418,long_haul_aircraft


# Export data

In [41]:
wiki_aircrafts_names = all_aircrafts.Model.unique()
wiki_aircrafts_names.sort()
path = os.path.join(".", "data", "wikipedia", "wiki_aircrafts_names.csv")
pd.Series(wiki_aircrafts_names).to_csv(path, header=True)

In [42]:
path = os.path.join(".", "data", "planespotter", "planespotter_aircratfs_dictionnary.csv")
planespotter_aircrafts_dictionnary.to_csv(path)

We will now nanualy go and create the dataset that matches planespotters aicraft model names to the names of the wikipedia aircrafts. It is not worth automating this step since the dataset that we need to produce is not that large. Doing it by hand also allows use to ensure that there is no mistakes in the pairing.

# Import data once manually updated

In [43]:
path = os.path.join(".", "data", "planespotter", "planespotter_aircratfs_dictionnary_corrected.csv")
planespotter_aircrafts_dictionnary_corrected = pd.read_csv(path)

We had to make the dataset that links the planespottes naming ot the wikipedia names manually.

# Merge all_aircrafts and the dictionnary

In [44]:
all_aircrafts = all_aircrafts.set_index('Model')
all_aircrafts.head()

,Seats,First flight,Fuel burn kg/km,Fuel per seat L/100km,Sector km,CO2 per seat kg/100km,CO2 kg/km,Aircraft type
Model,,,,,,,,
Antonov An-148 (241 nmi),89,2004,4.23,5.95,560.0,13.187973,13.05378,commuter_aircraft
Antonov An-158 (241 nmi),99,2010,4.34,5.47,560.0,12.124069,13.39324,commuter_aircraft
ATR 42-500,48,1995,1.26,3.15,560.0,6.981868,3.88836,commuter_aircraft
ATR 72-500,70,1997,1.42,2.53,560.0,5.607659,4.38212,commuter_aircraft
Beechcraft 1900D (226 nm),19,1982,1.00,6.57,560.0,14.562182,3.08600,commuter_aircraft


In [45]:
# Join planespotter_aircrafts_dictionnary_corrected and all_aircrafts
planespotter_aircrafts_dictionnary_corrected = planespotter_aircrafts_dictionnary_corrected.rename(columns={'Wiki name':'Model'})[["Model", "Planespotter name"]]
planespotter_aircrafts_dictionnary_corrected = planespotter_aircrafts_dictionnary_corrected.set_index('Model')
planespotter_aircrafts_dictionnary_corrected.head()

,Planespotter name
Model,
ATR 42-500,ATR 42
ATR 72-500,ATR 72
Airbus A220-100,Airbus A220-100
Airbus A220 100,Airbus A220-100
Airbus A220-300,Airbus A220-300


Our Model collum which is an idex is actually our wikipedia name. It is simply named model to facilitate the join opperation with the planespotter dataset.

In [46]:
all_aircrafts_corrected = all_aircrafts.join(planespotter_aircrafts_dictionnary_corrected, how = 'inner')

In [47]:
all_aircrafts_corrected = all_aircrafts_corrected.reset_index().drop(['Model'], axis=1).rename(columns={"Planespotter name": "Model"}).set_index("Model")

In [48]:
all_aircrafts_corrected.head()

,Seats,First flight,Fuel burn kg/km,Fuel per seat L/100km,Sector km,CO2 per seat kg/100km,CO2 kg/km,Aircraft type
Model,,,,,,,,
ATR 42,48,1995,1.26,3.15,560.0,6.981868,3.88836,commuter_aircraft
ATR 72,70,1997,1.42,2.53,560.0,5.607659,4.38212,commuter_aircraft
Airbus A220-100,115,2013,2.80,3.07,1100.0,6.804551,8.64080,regional_aircraft
Airbus A220-300,140,2015,3.10,2.75,1100.0,6.095282,9.56660,regional_aircraft
Airbus A220-100,125,2013,2.57,2.57,930.0,5.696318,7.93102,regional_aircraft


In [49]:
all_aircrafts_corrected.index.is_unique

False

Ohh dear, we do not have unique aircraft model names. This is bothersome. Let's fix this issue by averaging the data that we have. This should give use a good estimate of the fuel comsumption and co2 production without having to inspect all the details. As we can see above the issue is that aircrafts of the same model can be equiped with different number of seats. This means that the overal mass of the aircraft is slightly different which explains the slight difference in fuel burn / km observed.

In [50]:
all_aircrafts_corrected = all_aircrafts_corrected.reset_index().groupby("Model").mean()
all_aircrafts_corrected.head()

,Seats,First flight,Fuel burn kg/km,Fuel per seat L/100km,Sector km,CO2 per seat kg/100km,CO2 kg/km
Model,,,,,,,
ATR 42,48.000000,1995.0,1.2600,3.1500,560.0,6.981868,3.888360
ATR 72,70.000000,1997.0,1.4200,2.5300,560.0,5.607659,4.382120
Airbus A220-100,121.666667,2013.0,2.5500,2.6400,1310.0,5.851470,7.869300
Airbus A220-300,148.750000,2015.0,2.7025,2.2075,1457.5,4.892849,8.339915
Airbus A318-100,124.000000,1995.0,2.9300,2.9500,1900.0,6.538575,9.041980


In [51]:
all_aircrafts_corrected.index.is_unique

True

Problem fixed!

In [52]:
planespotter_aircrafts = planespotter_aircrafts.rename(columns={"Aircraft Type":"Model"}).reset_index().drop('index', axis = 1).set_index('Model')
planespotter_aircrafts.head()

,Aircraft Manufacturer,Operator,Operator ID,Operator Category,Operator Country,Operator Parent/Group,Parent/Group ID,Parent/Group Country,First Operator,First Operator ID,...,Build Country,First Customer Delivery Date,Delivery Date Operator,Exit Date Operator,Nr. of Engines,Engine Manufacturer,Engine Type,Config (Pax/Combi/Cargo/Other),Seat Total,Age (Years)
Model,,,,,,,,,,,,,,,,,,,,,
Airbus A300B1,Airbus,Airbus Industrie,1336,Manufacturer,France,NaN,NaN,NaN,NaN,NaN,...,France,NaN,1972-01-01,NaN,2,General Electric,CF6-50,Other,0,1.7
Airbus A300B1,Airbus,TEA - Trans European Airlines,294,Airline,Belgium,NaN,NaN,NaN,TEA - Trans European Airlines,294.0,...,France,1974-11-01,1974-11-01,NaN,2,General Electric,CF6-50,NaN,0,30.4
Airbus A300B2,Airbus,SA Novespace,7216,Other,France,NaN,NaN,NaN,SA Novespace,7216.0,...,France,2005-01-01,2005-01-01,NaN,2,General Electric,CF6-50,NaN,0,46.4
Airbus A300B2,Airbus,Air Inter,1309,Airline,France,NaN,NaN,NaN,Air Inter,1309.0,...,France,1977-01-01,1977-01-01,NaN,2,General Electric,CF6-50,NaN,0,19.4
Airbus A300B2,Airbus,Air France,1306,Airline,France,Air France-KLM,11321.0,France,Air France,1306.0,...,France,1974-05-01,1974-05-01,NaN,2,General Electric,CF6-50,NaN,0,23.1


In [53]:
aircrafts_airlines = all_aircrafts_corrected.join(planespotter_aircrafts, how = 'inner')

In [54]:
aircrafts_airlines.columns

Index(['Seats', 'First flight', 'Fuel burn kg/km', 'Fuel per seat L/100km',
       'Sector km', 'CO2 per seat kg/100km', 'CO2 kg/km',
       'Aircraft Manufacturer', 'Operator', 'Operator ID', 'Operator Category',
       'Operator Country', 'Operator Parent/Group', 'Parent/Group ID',
       'Parent/Group Country', 'First Operator', 'First Operator ID',
       'First Operator Category', 'First Operator Country', 'Status',
       'Build Date', 'Build Country', 'First Customer Delivery Date',
       'Delivery Date Operator', 'Exit Date Operator', 'Nr. of Engines',
       'Engine Manufacturer', 'Engine Type', 'Config (Pax/Combi/Cargo/Other)',
       'Seat Total', 'Age (Years)'],
      dtype='object')

In [55]:
aircrafts_airlines.groupby("Operator").mean()

,Seats,First flight,Fuel burn kg/km,Fuel per seat L/100km,Sector km,CO2 per seat kg/100km,CO2 kg/km,Operator ID,Parent/Group ID,First Operator ID,Nr. of Engines,Seat Total,Age (Years)
Operator,,,,,,,,,,,,,
'Ohana by Hawaiian,57.428571,1995.857143,1.328571,2.884286,560.000000,6.392921,4.099971,10163.0,NaN,1594.428571,2.000000,27.428571,20.814286
19th Hole,70.000000,1997.000000,1.420000,2.530000,560.000000,5.607659,4.382120,10259.0,NaN,1042.000000,2.000000,0.000000,22.000000
21 Air,199.333333,1984.000000,4.923333,3.113333,5833.333333,6.900597,15.193407,10467.0,NaN,380.000000,2.000000,0.000000,32.750000
2Excel Aviation,126.000000,1984.000000,3.490000,3.460000,939.000000,7.668972,10.770140,10108.0,NaN,2229.500000,2.000000,31.000000,27.550000
328 Support Services,31.500000,1991.000000,1.150000,4.555000,830.000000,10.096003,3.548900,9935.0,NaN,3889.250000,2.000000,7.750000,23.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
flyadeal,150.000000,1987.000000,3.020000,2.520000,2942.000000,5.585494,9.319720,11044.0,NaN,11044.000000,2.000000,186.000000,1.627273
flydubai,162.579235,2002.573770,3.263060,2.502459,2012.349727,5.546615,10.069803,8579.0,NaN,8579.000000,2.000000,175.016393,4.336066
flynas,161.888889,1992.333333,3.435278,2.605833,2796.722222,5.775741,10.601267,10290.0,NaN,6354.638889,2.111111,121.000000,11.242857


In [56]:
(aircrafts_airlines[aircrafts_airlines["Operator Category"] == "Manufacturer"])\
    .groupby(["Aircraft Manufacturer"])\
    .count()[["Seats"]].rename(columns={"Seats": "Number of planes"})

,Number of planes
Aircraft Manufacturer,
ATR,17
Airbus,26
Boeing,49
Bombardier,23
Dornier,11
Embraer,23
Eurocopter,9
Lockheed Martin,1


## Let's look at the largest airlines in the world

In [57]:
aircrafts_airlines["Operator Category"].unique()

array(['Governmental', 'Airline', 'Private', 'Air Force', 'Manufacturer',
       'Leasing Company', nan, 'Brand', 'Trustee', 'Other',
       'Business Airline'], dtype=object)

Ok we have a few different types of operator. We will focus on Airline.

We will consider a large airline to have at least 30 aircrafts

In [58]:
aircrafts_airlines.columns

Index(['Seats', 'First flight', 'Fuel burn kg/km', 'Fuel per seat L/100km',
       'Sector km', 'CO2 per seat kg/100km', 'CO2 kg/km',
       'Aircraft Manufacturer', 'Operator', 'Operator ID', 'Operator Category',
       'Operator Country', 'Operator Parent/Group', 'Parent/Group ID',
       'Parent/Group Country', 'First Operator', 'First Operator ID',
       'First Operator Category', 'First Operator Country', 'Status',
       'Build Date', 'Build Country', 'First Customer Delivery Date',
       'Delivery Date Operator', 'Exit Date Operator', 'Nr. of Engines',
       'Engine Manufacturer', 'Engine Type', 'Config (Pax/Combi/Cargo/Other)',
       'Seat Total', 'Age (Years)'],
      dtype='object')

In [70]:
big_airlines = aircrafts_airlines.copy()
big_airlines = big_airlines[big_airlines["Operator Category"] == 'Airline']
big_airlines = big_airlines[big_airlines["Status"] == "Active"]
big_airlines = big_airlines.groupby(['Operator', 'Operator ID']).count()
big_airlines = big_airlines[["Seats"]].rename(columns={"Seats": "Nb planes"})
big_airlines = big_airlines.sort_values("Nb planes", ascending = False)

# TODO comment at large airlines

In [77]:
big_airlines = big_airlines[big_airlines['Nb planes'] > 30].reset_index()

In [78]:
big_airlines_list = big_airlines.Operator.tolist()
len(big_airlines_list)

169

Let's have a look if we have many manufacturers of aircrafts in our dataset.

We have narrowed down all of our airlines almost 2000 from the, to the 10% that have the most aircrafts. We also removed the manufactrers from the list but these are few compared to the number of small airlines.

In [79]:
plot_airlines = aircrafts_airlines.copy()
plot_airlines = plot_airlines.reset_index()[["Operator", "CO2 per seat kg/100km"]]
plot_airlines = plot_airlines[[x in big_airlines_list for x in plot_airlines["Operator"]]]
plot_airlines = plot_airlines.groupby('Operator').mean().reset_index()
plot_airlines.head()

,Operator,CO2 per seat kg/100km
0,Aegean Airlines,5.582835
1,Aer Lingus,5.840812
2,Aeroflot - Russian Airlines,5.776314
3,Aerolineas Argentinas,6.072240
4,Aeromexico,5.764839


In [117]:
px.bar(data_frame = plot_airlines.sort_values("CO2 per seat kg/100km"), x="Operator", y="CO2 per seat kg/100km")

In [81]:
big_polluters = plot_airlines[plot_airlines["CO2 per seat kg/100km"] > 8]

After a bit of reasearch we have noticed that most of these airlines are subsidaries of larger airlines but are the regional sector. For example *Lufthansa CityLine* is a compagny that belongs entirely to *Lufthansa* and opperates flights on there behalf.

Let's have a closer look at what is hapenning with the *Lufthansa CityLine*

In [82]:
lufthansa_fleet = aircrafts_airlines[aircrafts_airlines['Operator'] == "Lufthansa CityLine"]
lufthansa_fleet= lufthansa_fleet.reset_index().groupby('Model').count()

In [83]:
lufthansa_fleet

,Seats,First flight,Fuel burn kg/km,Fuel per seat L/100km,Sector km,CO2 per seat kg/100km,CO2 kg/km,Aircraft Manufacturer,Operator,Operator ID,...,Build Country,First Customer Delivery Date,Delivery Date Operator,Exit Date Operator,Nr. of Engines,Engine Manufacturer,Engine Type,Config (Pax/Combi/Cargo/Other),Seat Total,Age (Years)
Model,,,,,,,,,,,,,,,,,,,,,
Airbus A319-100,3,3,3,3,3,3,3,3,3,3,...,3,3,3,0,3,3,3,3,3,3
Bombardier CRJ-100,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,0
Bombardier CRJ-700,9,9,9,9,9,9,9,9,9,9,...,9,9,9,0,9,9,9,9,9,6
Bombardier CRJ-900,35,35,35,35,35,35,35,35,35,35,...,35,35,35,0,35,35,35,35,35,35
Embraer ERJ-190,9,9,9,9,9,9,9,9,9,9,...,9,9,9,0,9,9,9,9,9,9
Embraer ERJ-195,3,3,3,3,3,3,3,3,3,3,...,3,3,3,0,3,3,3,3,3,3


We can notice that our hypothesis about regional aircrafts is confirmed. Most of *Lufthnasa CityLine* fleet are **Bombardier CRJ-900**.

In [84]:
aircrafts_used_by_big_polluters = aircrafts_airlines.reset_index().set_index("Operator")\
.loc[big_polluters.Operator.values].groupby("Model").\
count()[["Seats"]].rename(columns={"Seats": "Number of aircrafts"})\
.join(all_aircrafts_corrected, how ="inner")

In [85]:
aircrafts_used_by_big_polluters

,Number of aircrafts,Seats,First flight,Fuel burn kg/km,Fuel per seat L/100km,Sector km,CO2 per seat kg/100km,CO2 kg/km
Model,,,,,,,,
ATR 42,23,48.0,1995.0,1.260,3.150,560.0,6.981868,3.88836
ATR 72,7,70.0,1997.0,1.420,2.530,560.0,5.607659,4.38212
Airbus A319-100,3,124.0,1995.0,2.930,2.950,1900.0,6.538575,9.04198
Airbus A320-200,11,150.0,1987.0,3.020,2.520,2942.0,5.585494,9.31972
Bombardier CRJ-100,20,50.0,1991.0,2.040,5.090,814.5,11.281812,6.29544
Bombardier CRJ-1000,14,100.0,2009.0,2.660,3.330,930.0,7.380832,8.20876
Bombardier CRJ-200,475,50.0,1995.0,1.990,4.960,815.0,10.993671,6.14114
Bombardier CRJ-700,189,70.0,1999.0,2.700,4.805,811.5,10.650119,8.33220
Bombardier CRJ-900,268,88.0,2001.0,3.125,4.425,810.5,9.807862,9.64375


In [86]:
px.bar(data_frame=aircrafts_used_by_big_polluters.sort_values("CO2 per seat kg/100km").reset_index(),
             x = "Model",
             y = "Number of aircrafts",
             hover_name="Model",
             color= "CO2 per seat kg/100km",
             color_discrete_sequence=np.repeat(px.colors.sequential.Inferno, 3),
             title="Jet's used for regional flights. Colors indicate CO2 per seat",
            )

In [87]:
px.box(
    aircrafts_used_by_big_polluters.reset_index(),
    x="CO2 per seat kg/100km",
    y= "Sector km",
    points="all",
    hover_data = ["CO2 kg/km", "CO2 per seat kg/100km"],
    hover_name ="Model")

In [89]:
px.histogram(
    aircrafts_used_by_big_polluters,
    x = "Sector km",
    y ="Number of aircrafts",
    histfunc="sum",
    nbins=30,
    marginal="rug")

Ok let's state the what we can conclude from the above few graphs. We can see that most of the aircrafts that pollute a lot are used for short ranges. The main culpride is the Bombardier CRJ-200.

<img src="https://www.aircraftcompare.com/wp-content/uploads/2009/01/Bombardier-CRJ-200-Air-Wisconsin.jpg" alt="logo" />

These small jets are used to connect small airports to main hubs.

In [90]:
aircrafts_airlines.reset_index().groupby(["Operator", "Model"]).mean()

Seats  First flight  \
Operator            Model                                          
'Ohana by Hawaiian  ATR 42               48.000000        1995.0   
                    ATR 72               70.000000        1997.0   
19th Hole           ATR 72               70.000000        1997.0   
21 Air              Boeing 767-200      199.333333        1984.0   
2Excel Aviation     Boeing 737-300      126.000000        1984.0   
...                                            ...           ...   
flynas              Airbus A320neo      154.000000        2015.0   
                    Boeing 747-400      416.000000        1988.0   
                    Embraer ERJ-190     114.000000        2004.0   
go! (Mesa Airlines) Bombardier CRJ-200   50.000000        1995.0   
iXAir               Eurocopter EC135     37.000000        1998.0   

                                        Fuel burn kg/km  \
Operator            Model                                 
'Ohana by Hawaiian  ATR 42                     1.260000   
                    ATR 72                     1.420000   
19th Hole           ATR 72                     1.420000   
21 Air              Boeing 767-200             4.923333   
2Excel Aviation     Boeing 737-300             3.490000   
...                                                 ...   
flynas              Airbus A320neo             2.790000   
                    Boeing 747-400            10.940000   
                    Embraer ERJ-190            3.240000   
go! (Mesa Airlines) Bombardier CRJ-200         1.990000   
iXAir               Eurocopter EC135           1.440000   

                                        Fuel per seat L/100km    Sector km  \
Operator            Model                                                    
'Ohana by Hawaiian  ATR 42                           3.150000   560.000000   
                    ATR 72                           2.530000   560.000000   
19th Hole           ATR 72                           2.530000   560.000000   
21 Air              Boeing 767-200                   3.113333  5833.333333   
2Excel Aviation     Boeing 737-300                   3.460000   939.000000   
...                                                       ...          ...   
flynas              Airbus A320neo                   2.250000  1220.000000   
                    Boeing 747-400                   3.290000  7492.000000   
                    Embraer ERJ-190                  3.540000  1124.000000   
go! (Mesa Airlines) Bombardier CRJ-200               4.960000   815.000000   
iXAir               Eurocopter EC135                 4.860000  1104.000000   

                                        CO2 per seat kg/100km  CO2 kg/km  \
Operator            Model                                                  
'Ohana by Hawaiian  ATR 42                           6.981868   3.888360   
                    ATR 72                           5.607659   4.382120   
19th Hole           ATR 72                           5.607659   4.382120   
21 Air              Boeing 767-200                   6.900597  15.193407   
2Excel Aviation     Boeing 737-300                   7.668972  10.770140   
...                                                       ...        ...   
flynas              Airbus A320neo                   4.987049   8.609940   
                    Boeing 747-400                   7.292173  33.760840   
                    Embraer ERJ-190                  7.846290   9.998640   
go! (Mesa Airlines) Bombardier CRJ-200              10.993671   6.141140   
iXAir               Eurocopter EC135                10.772025   4.443840   

                                        Operator ID  Parent/Group ID  \
Operator            Model                                              
'Ohana by Hawaiian  ATR 42                  10163.0              NaN   
                    ATR 72                  10163.0              NaN   
19th Hole           ATR 72                  10259.0              NaN   
21 Air              Boeing 7

In [91]:
airlines_country = aircrafts_airlines.copy()
airlines_country = airlines_country[airlines_country["Operator Category"] == 'Airline']
airlines_country = airlines_country.groupby('Operator Country').count()
airlines_country = airlines_country[["Seats"]].rename(columns={"Seats": "Nb planes"})
airlines_country = airlines_country.sort_values("Nb planes", ascending = False)
airlines_country

,Nb planes
Operator Country,
United States,7347
China,4052
United Kingdom,1024
Russian Federation,879
India,795
...,...
Solomon Islands,1
Greenland,1
Guinea,1


In [92]:
airlines_country = airlines_country[airlines_country['Nb planes'] > 30].reset_index()
airlines_country_list = airlines_country['Operator Country'].tolist()
airlines_country

,Operator Country,Nb planes
0,United States,7347
1,China,4052
2,United Kingdom,1024
3,Russian Federation,879
4,India,795
...,...,...
77,Sri Lanka,38
78,Belarus,33
79,Uzbekistan,33
80,Cambodia,32


In [93]:
fig = px.choropleth(
    airlines_country,
    locations = "Operator Country",
    locationmode = "country names",
    color = "Nb planes",
    hover_name="Operator Country",
    projection = "natural earth",
    color_continuous_scale = px.colors.sequential.OrRd,
    title = "Planes per country")
fig.show()

We see that the US, China, and the UK are dominating which countries have the most aircrafts. Let's look at the world without these 3 mega countries.

In [94]:
fig = px.choropleth(
    airlines_country.iloc[3:],
    locations = "Operator Country",
    locationmode = "country names",
    color = "Nb planes",
    hover_name = "Nb planes", 
    projection = "natural earth",
    color_continuous_scale = px.colors.sequential.OrRd,
    title = "Planes per country without the largest 3")
fig.show()

## CO2 emmisions average by country

We will look at the average CO2 emmissions per country.

In [95]:
plot_country = aircrafts_airlines.copy()
plot_country = plot_country.reset_index()[["Operator Country", "CO2 per seat kg/100km"]]
plot_country = plot_country[[x in airlines_country_list for x in plot_country["Operator Country"]]]
plot_country = plot_country.groupby('Operator Country').mean().reset_index()
plot_country.head()

,Operator Country,CO2 per seat kg/100km
0,Algeria,6.091895
1,Argentina,6.733893
2,Australia,6.229032
3,Austria,6.889562
4,Azerbaijan,6.653428


In [99]:
px.histogram(plot_country.sort_values("CO2 per seat kg/100km"), x = "Operator Country", y = "CO2 per seat kg/100km", histfunc="sum")

In [100]:
"""
Possible colours for our graphs

Brwnyl
Bluered
"""

'\nPossible colours for our graphs\n\nBrwnyl\nBluered\n'

In [103]:
fig = px.choropleth(
    plot_country,
    locations = "Operator Country",
    locationmode = "country names",
    color = "CO2 per seat kg/100km",
    hover_name = "CO2 per seat kg/100km", 
    projection = "natural earth",
    color_continuous_scale = px.colors.sequential.OrRd,
    title = "Average CO2 per seat kg/100km")
fig.show()

Given the maps above we can see that data is lacking for a proper analysis of the africain continent. For the rest of the world the maps above make use hypothesise 2 things. First of all we can see that western countries (ie: USA, Canada, Europe) have a tendency to pollute more than. We hypothesis that this is due to a higher income than the rest of the world allowing people to travell more short distances. Poorer countries probably do not have a lot of internal flights since most of the population cannot afford to fly.

Let's first of all check if the hypothesis about the income is correct

In [104]:
income_by_country = pd.read_csv(os.path.join(".","data","gapminder","income_per_person_gdppercapita_ppp_inflation_adjusted.csv.xz"))

In [105]:
income_by_country[["country", "2019"]]

,country,2019
0,Afghanistan,1950
1,Albania,13000
2,Algeria,13700
3,Andorra,53200
4,Angola,5760
...,...,...
188,Venezuela,14000
189,Vietnam,6870
190,Yemen,2500
191,Zambia,3980


In [107]:
fig = px.choropleth(
    income_by_country[["country", "2019"]],
    locations="country",
    locationmode="country names",
    color="2019",
    projection="natural earth",
    color_continuous_scale=px.colors.sequential.OrRd)

fig.show()

We can see that our hypotheis above appears to be true

## Small polluters

In [108]:
small_polluters = plot_airlines[plot_airlines["CO2 per seat kg/100km"] < 5.55]

In [109]:
small_polluters

,Operator,CO2 per seat kg/100km
7,Air Arabia,5.517454
20,AirAsia,5.410837
67,Frontier Airlines,5.494874
70,GoAir,5.152137
78,IndiGo,5.323199
79,Interjet,5.421159
96,Loong Air,5.526228
107,Pegasus,5.457661
132,Spring Airlines,5.507436
152,VietJetAir,5.415886


From looking at this list we can see that most of the airlines that are listed above are lowcost airlines. This could be explained by the fact that lowcost airlines tend to sqeeze more seats into an aircraft, will reducing what is included with the flight which would probably reduce the total mass of the plane. This question would require further reasearch.

In [110]:
aircrafts_used_by_small_polluters = aircrafts_airlines.reset_index().set_index("Operator")\
.loc[small_polluters.Operator.values].groupby("Model").\
count()[["Seats"]].rename(columns={"Seats": "Number of aircrafts"})\
.join(all_aircrafts_corrected, how ="inner")

In [111]:
aircrafts_used_by_small_polluters

,Number of aircrafts,Seats,First flight,Fuel burn kg/km,Fuel per seat L/100km,Sector km,CO2 per seat kg/100km,CO2 kg/km
Model,,,,,,,,
ATR 72,29,70.000000,1997.0,1.420000,2.530000,560.000000,5.607659,4.382120
Airbus A318-100,11,124.000000,1995.0,2.930000,2.950000,1900.000000,6.538575,9.041980
Airbus A319-100,24,124.000000,1995.0,2.930000,2.950000,1900.000000,6.538575,9.041980
Airbus A320-200,672,150.000000,1987.0,3.020000,2.520000,2942.000000,5.585494,9.319720
Airbus A320neo,343,154.000000,2015.0,2.790000,2.250000,1220.000000,4.987049,8.609940
Airbus A321-200,174,180.000000,1996.0,3.610000,2.500000,1900.000000,5.541165,11.140460
Airbus A321neo,82,192.000000,2015.0,3.300000,2.190000,1220.000000,4.854061,10.183800
Airbus A330-200,4,258.333333,1997.0,6.000000,2.933333,6166.666667,6.501634,18.516000
Airbus A350-900,14,315.000000,2013.0,6.550000,2.600000,10662.000000,5.762812,20.213300


In [112]:
px.bar(data_frame=aircrafts_used_by_small_polluters.sort_values("CO2 per seat kg/100km").reset_index(),
             x = "Model",
             y = "Number of aircrafts",
             hover_name="Model",
             hover_data=["First flight"],
             color= "CO2 per seat kg/100km",
             color_discrete_sequence=np.repeat(px.colors.sequential.Inferno, 2),
             title="Jet's used by the airlines producing the least CO2. Colors indicate CO2 per seat",
            )

We can see that more recent aircrafts pollute less. The Neo and Max family at Airbus and Boeing respectevely are the newest kind of aircrafts that are being produced.

In [113]:
px.box(
    aircrafts_used_by_small_polluters.reset_index(),
    x="CO2 per seat kg/100km",
    y= "Sector km",
    points="all",
    hover_data = ["CO2 kg/km", "CO2 per seat kg/100km"],
    hover_name ="Model",
    title = "Distance that the aircraft used by the least polluting airlines can fly vs CO2 per seat kg/100km")

In [114]:
aircrafts_used_by_small_polluters['Eco'] = "True"

In [115]:
aircrafts_used_by_big_polluters["Eco"] = "False"
pd.concat([aircrafts_used_by_big_polluters, aircrafts_used_by_small_polluters])

,Number of aircrafts,Seats,First flight,Fuel burn kg/km,Fuel per seat L/100km,Sector km,CO2 per seat kg/100km,CO2 kg/km,Eco
Model,,,,,,,,,
ATR 42,23,48.000000,1995.0,1.260000,3.150000,560.000000,6.981868,3.888360,False
ATR 72,7,70.000000,1997.0,1.420000,2.530000,560.000000,5.607659,4.382120,False
Airbus A319-100,3,124.000000,1995.0,2.930000,2.950000,1900.000000,6.538575,9.041980,False
Airbus A320-200,11,150.000000,1987.0,3.020000,2.520000,2942.000000,5.585494,9.319720,False
Bombardier CRJ-100,20,50.000000,1991.0,2.040000,5.090000,814.500000,11.281812,6.295440,False
Bombardier CRJ-1000,14,100.000000,2009.0,2.660000,3.330000,930.000000,7.380832,8.208760,False
Bombardier CRJ-200,475,50.000000,1995.0,1.990000,4.960000,815.000000,10.993671,6.141140,False
Bombardier CRJ-700,189,70.000000,1999.0,2.700000,4.805000,811.500000,10.650119,8.332200,False
Bombardier CRJ-900,268,88.000000,2001.0,3.125000,4.425000,810.500000,9.807862,9.643750,False


In [116]:
px.histogram(
    pd.concat([aircrafts_used_by_big_polluters, aircrafts_used_by_small_polluters]),
    x = "Sector km",
    y ="Number of aircrafts",
    color = "Eco",
    histfunc="sum",
    nbins=80,
    marginal="rug",
    title = "Average range of the big polluter aircrafts")

We can see from the histogram above that airlines that are polluting less are operating flights with aircrafts that are designed to fly further.

We can conclude from this analysis that we have found significant differences between local airlines and low cost airlines. Flying with a low cost will most likely reduce your CO2 emissions compared to flying with a regional jet. The reason of being of these 2 types of flights is entirely different. The purpose of regional flights is to allow passengers to stop over in one of the main hubs of the airline before continuing on with another flight. So these flights can be more expensive (burn more fuel) than flights operated by a low-cost airline. 

Low-cost are on the other hand trying to minimize the operating costs. Since fuel accounts for approximately 25% of operating costs (according to https://www.statista.com/statistics/591285/aviation-industry-fuel-cost/) low cost airlines have a large incentive to opperate fleets of aircrafts that burn less fuel.